In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

if __name__ =="__main__":
    spark = SparkSession.builder.master("local[*]").getOrCreate()

    schema = StructType([
        StructField("artist_name", StringType(), True),
        StructField("album_name", StringType(), True),
        StructField("track_number", DoubleType(), True),
        StructField("song_id", DoubleType(), True),
        StructField("song_name", StringType(), True),
        StructField("line_number", DoubleType(), True),
        StructField("section_name", StringType(), True),
        StructField("line", StringType(), True),
        StructField("section_artist", StringType(), True)
    ])

    lyrics_df = (
        spark
        .readStream
        .format("csv")
        .schema(schema)
        .option("header", True)
        .load("./content")
    ) 

In [ ]:


    query = (
        lyrics_df
        .writeStream
        .format("console")
        .outputMode("append")
        .start()
    ).awaitTermination()


In [8]:
wordCountDf = (
        lyrics_df
        .select(
            explode(split(col("artist_name"), "\s+"))
            .alias("word")
        )
        .groupBy("word")
        .count()
    )

query = (
    wordCountDf
    .writeStream
    .format("console")
    .outputMode("complete")
    .start()
).awaitTermination()

spark.stop()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 